In [24]:
needle_model_name = "NeedleModel"
sequence_browser_name = "NeedleCapture_01_SequenceBrowser"
image_name = "Image_Image"
segmentation_name = "Segmentation"

output_folder = "c:\Temp"
filename_prefix = "01"

In [25]:
import datetime
import numpy as np
import os

In [26]:
browser_node = slicer.util.getFirstNodeByName(sequence_browser_name, className='vtkMRMLSequenceBrowserNode')
model_node = slicer.util.getFirstNodeByName(needle_model_name, className="vtkMRMLModelNode")
image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
segmentation_node = slicer.util.getFirstNodeByName(segmentation_name, className="vtkMRMLSegmentationNode")

if browser_node is None:
    logging.error("Could not find input browser node: {}".format(sequence_browser_name))
    raise

if model_node is None:
    logging.error("Could not find input model node: {}".format(needle_model_name))
    raise

if image_node is None:
    logging.error("Could not find input image node: {}".format(image_name))
    raise

if segmentation_node is None:
    logging.error("Could not find input segmentation node: {}".format(segmentation_name))
    raise


In [27]:
labelmap_name = "temp_labelmap"
labelmap_node = slicer.util.getFirstNodeByName(labelmap_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_node is None:
    labelmap_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_node.SetName(labelmap_name)

In [30]:
import vtkSegmentationCorePython as vtkSegmentationCore

def eraseCurrentSegmentation(selectedSegmentation):
    num_segments = selectedSegmentation.GetSegmentation().GetNumberOfSegments()
    for i in range(num_segments):
        segmentId = selectedSegmentation.GetSegmentation().GetNthSegmentID(i)
        import vtkSegmentationCorePython as vtkSegmentationCore
        try:
            labelMapRep = selectedSegmentation.GetBinaryLabelmapRepresentation(segmentId)
        except:
            labelMapRep = selectedSegmentation.GetBinaryLabelmapInternalRepresentation(segmentId)
        slicer.vtkOrientedImageDataResample.FillImage(labelMapRep, 0, labelMapRep.GetExtent())
        slicer.vtkSlicerSegmentationsModuleLogic.SetBinaryLabelmapToSegment(
            labelMapRep, selectedSegmentation, segmentId, slicer.vtkSlicerSegmentationsModuleLogic.MODE_REPLACE)

In [ ]:
browser_node.SelectFirstItem()

segmentation_logic = slicer.modules.segmentations.logic()

png_writer = vtk.vtkPNGWriter()

image_cast = vtk.vtkImageCast()
image_cast.SetOutputScalarTypeToUnsignedChar()
image_cast.Update()

eraseCurrentSegmentation(segmentation_node)
n = browser_node.GetNumberOfItems()
for i in range(n):
    if i > 10:
        break
    
    # input_array = slicer.util.array(input_image_node.GetID())
    
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmap_node, image_node)
    segmented_image = labelmap_node.GetImageData()
    
    img_file_name = filename_prefix + "_%04d_ultrasound" % i + ".png"
    ultrasound_fullname = os.path.join(output_folder, img_file_name)
    
    seg_file_name = filename_prefix + "_%04d_segmentation" % i + ".png"
    segmentation_fullname = os.path.join(output_folder, seg_file_name)
    
    png_writer.SetInputData(image_node.GetImageData())
    png_writer.SetFileName(ultrasound_fullname)
    png_writer.Update()
    png_writer.Write()
    
    image_cast.SetInputData(segmented_image)
    image_cast.Update()
    png_writer.SetInputData(image_cast.GetOutput())
    png_writer.SetFileName(segmentation_fullname)
    png_writer.Update()
    png_writer.Write()
    
    eraseCurrentSegmentation(segmentation_node)
    
    slicer.app.processEvents()
    browser_node.SelectNextItem()